# Reduceret echelonform

In [44]:
import numpy as np
from sympy import Matrix, latex

def row_echelon_form_operations(matrix, precision=6):
    # Convert the matrix to a SymPy matrix object
    sympy_matrix = Matrix(matrix)

    # Convert SymPy matrix to a NumPy array for numerical operations
    numpy_array = np.array(sympy_matrix.tolist(), dtype=float)
    # Track row operations for later retrieval
    operations = []

    # Apply Gaussian elimination to obtain the reduced row echelon form
    num_rows, num_cols = numpy_array.shape
    lead = 0
    for r in range(num_rows):
        if lead >= num_cols:
            break

        # Find a non-zero pivot element, swap rows if necessary
        if numpy_array[r, lead] == 0:
            for i in range(r + 1, num_rows):
                if numpy_array[i, lead] != 0:
                    numpy_array[[r, i], :] = numpy_array[[i, r], :]
                    operations.append(f"Swap R{r + 1} and R{i + 1}")
                    break

        # If all elements in the current column are zero, move to the next column
        if numpy_array[r, lead] == 0:
            lead += 1
            continue

        pivot = numpy_array[r, lead]
        numpy_array[r, :] /= pivot
        operations.append(f"Scale R{r + 1} by 1/{pivot}")

        for i in range(num_rows):
            if i != r:
                factor = numpy_array[i, lead]
                numpy_array[i, :] -= factor * numpy_array[r, :]
                operations.append(f"R{i + 1} -> R{i + 1} - {factor} * R{r + 1}")

        lead += 1

    # Round very small values to zero
    numpy_array[np.abs(numpy_array) < 10 ** (-precision)] = 0

    # Convert NumPy array back to SymPy matrix
    echelon_matrix = Matrix(numpy_array)

    # Convert the matrix to LaTeX format
    echelon_latex = latex(echelon_matrix)

    return echelon_latex, operations

# Matric som skal reduceres (hint: hvis du skal finde echelonform til et ligningssystem, så indsæt matricen du får EFTER at bruge np.stack)
matrix = [[1, 0, 3, 0],
          [0, 1, -8, 1],
          [0, 3, 0, 0]]

echelon_form, operations = row_echelon_form_operations(matrix)
print("Row Echelon Form:")
print(echelon_form)
print("Operations:")
for operation in operations:
    print(operation)

Row Echelon Form:
\left[\begin{matrix}1.0 & 0 & 0 & 0.375\\0 & 1.0 & 0 & 0\\0 & 0 & 1.0 & -0.125\end{matrix}\right]
Operations:
Scale R1 by 1/1.0
R2 -> R2 - 0.0 * R1
R3 -> R3 - 0.0 * R1
Scale R2 by 1/1.0
R1 -> R1 - 0.0 * R2
R3 -> R3 - 3.0 * R2
Scale R3 by 1/24.0
R1 -> R1 - 3.0 * R3
R2 -> R2 - -8.0 * R3


I forhold til resultaterne ovenfor, så er følgende betydningen: $\\$
Scale betyder skalar multiplikation (matrix[0, :] *= 1/1.0) $\\$
R3 -> R3 - 2.0 * R1 betyder R3 skal laves om på med hensyn det til på højre side $\\$
Swap betyder der skal byttes om på de to rækker (matrix[ [1, 2], :] = matrix[ [2, 1], :])

In [ ]:
# Det er vigtigt at tilføje dtype=float, ellers fungerer 1/1.0 ikke
matrix = np.array([[1, 1, 2, 1, 0], [1, 1, 3, 2, 1], [2, 2, 5, 4, 1]], dtype=float)

matrix[0, :] *= 1/3.0
matrix[1, :] = matrix[1, :] - 1*matrix[0, :]
matrix[2, :] = matrix[2, :] - 2*matrix[0, :]
matrix[1, :] *= 1/1.0
matrix[0, :] = matrix[0, :] - 2*matrix[1, :]
matrix[2, :] = matrix[2, :] - 1*matrix[1, :]
matrix[2, :] *= 1/1.0
matrix[0, :] = matrix[0, :] - (-2)*matrix[2, :]
matrix[1, :] = matrix[1, :] - 1*matrix[2, :]
print(matrix)

# Anvendelse

Reduced row echelonform bruges til flere ting indenfor Numerisk lineær algebra. Herunder til at løse lineære ligningssystemer, nulrummet, søjlerummet m.m. $\\$
Lige nogle generelle informationer om, hvorfor man bruger reduced row echelonform. Reduced row echelonform kan f.x. bruges til at finde pivot elementer og frie variabler. Vi har derfor brugt den i aflevering 3, hvor vi skulle finde frem til én løsning af et ligningssystem, som afhænger af frie variabler. Pivot elementer er det første 1 tal i rækken, så i vores tilfælde ovenfor, er det $x_0$ i række 0, $x_2$ i række 1 og $x_3$ i række 2. Pivot søjlerne er derfor søjle 0, 2 og 3, da det er de søjler, hvor pivot elementerne befinder sig i. Variablerne i de ikke-pivot søjler er frie variabler, så i vores tilfælde er det $x_1$ og $x_4$. Hvis sidste søjle ville have været en pivot sølje, ville vores system ikke have en løsning.

**Find Nulrummet (eksempel ovenfor)** 

Nulrummet angives som N(A), hvor A er vores matrix. Først vil vi altid finde reduced row echelonform, som vi har gjort ovenfor. Derefter kan vi opstille et ligningssystem:
$$
    x_0 + x_1 - 2x_4 = 0, \\
    x_2 + x_4 = 0, \\
    x_3 = 0
$$
Vi har allerede fundet, at vores frie variabler er $x_1, x_4$. En generel løsning vil altså være:
$$
\begin{bmatrix}
    x_0 \\
    x_1 \\
    x_2 \\
    x_3 \\
    x_4
\end{bmatrix} =
\begin{bmatrix}
    -x_1 + 2x_4 \\
    x_1 \\
    -x_4 \\
    0 \\
    x_4
\end{bmatrix} =
x_1 \begin{bmatrix}
    -1 \\
    1 \\
    0 \\
    0 \\
    0
\end{bmatrix}
x_4 \begin{bmatrix}
    2 \\
    0 \\
    -1 \\
    0 \\
    1
\end{bmatrix}
$$